# Visual Analytics - Text Mining Lord of the Rings Movie Scripts

In [58]:
# Setup
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import csv

In [59]:
# load data - https://www.kaggle.com/paultimothymooney/lord-of-the-rings-data
scripts = pd.read_csv('data/lotr_scripts.csv', index_col=0)
chars = pd.read_csv('data/lotr_characters.csv')

In [60]:
scripts.head(3)

,char,dialog,movie
0,DEAGOL,"Oh Smeagol Ive got one! , Ive got a fish Smeag...",The Return of the King
1,SMEAGOL,"Pull it in! Go on, go on, go on, pull it in!",The Return of the King
2,DEAGOL,Arrghh!,The Return of the King


In [61]:
chars.head(3)

,birth,death,gender,hair,height,name,race,realm,spouse
0,NaN,NaN,Female,NaN,NaN,Adanel,Men,NaN,Belemir
1,TA 2978,"February 26 ,3019",Male,Dark (book) Light brown (movie),NaN,Boromir,Men,NaN,NaN
2,NaN,"March ,3019",Male,NaN,NaN,Lagduf,Orcs,NaN,NaN


In [103]:
# load books.txt https://github.com/tianyigu/Lord_of_the_ring_project/tree/master/LOTR_code/lotr_script_scripy/lotr
# read textfile
lotr1 = pd.read_csv('data/book1.txt', sep='\n', header=None, encoding='utf-8', quoting=csv.QUOTE_NONE)
lotr2 = pd.read_csv('data/book2.txt', sep='\n', header=None, encoding='utf-8', quoting=csv.QUOTE_NONE)
lotr3 = pd.read_csv('data/book3.txt', sep='\n', header=None, encoding='utf-8', quoting=csv.QUOTE_NONE)

# change column name to 'text'
lotr1.columns = ['text']
lotr2.columns = ['text']
lotr3.columns = ['text']

In [104]:
# lotr1: remove title, contents, foreword and prologue
idx = lotr1[lotr1['text'].str.contains('\* BOOK I \*')].index 
lotr1 = lotr1.iloc[idx[0]:, :]

# lotr2: remove title
idx = lotr2[lotr2['text'].str.contains('\* BOOK III \*')].index 
lotr2 = lotr2.iloc[idx[0]:, :]

# lotr3: remove title
idx = lotr3[lotr3['text'].str.contains('\* BOOK V \*')].index 
lotr3 = lotr3.iloc[idx[0]:, :]

In [105]:
# combine books into one dataframe
lotr = pd.concat([lotr1, lotr2, lotr3]).reset_index(drop=True)

In [108]:
# Add book and chapter information
lotr['book'] = lotr['text'].str.extract(r'(\* BOOK .* \*)', expand=False)
lotr['chapter'] = lotr['text'].str.extract(r'(^Chapter .*)', expand=False)

# fill empty rows with last available value
lotr['book'] = lotr['book'].fillna(method='ffill')
lotr['chapter'] = lotr['chapter'].fillna(method='ffill')

lotr

,text,book,chapter
0,* BOOK I *,* BOOK I *,NaN
1,Chapter 1 . A Long-expected Party,* BOOK I *,Chapter 1 . A Long-expected Party
2,When Mr. Bilbo Baggins of Bag End announced th...,* BOOK I *,Chapter 1 . A Long-expected Party
3,celebrating his eleventy -first birthday with ...,* BOOK I *,Chapter 1 . A Long-expected Party
4,"magnificence, there was much talk and exciteme...",* BOOK I *,Chapter 1 . A Long-expected Party
...,...,...,...
38524,1 See Appendix F under Ents,* BOOK VI *,Chapter 9 . The Grey Havens
38525,1 Every month in the Shire-calendar had 30 days.,* BOOK VI *,Chapter 9 . The Grey Havens
38526,"1 See App. F, 520.",* BOOK VI *,Chapter 9 . The Grey Havens
38527,( There were thirty days in March (or Rethe) i...,* BOOK VI *,Chapter 9 . The Grey Havens


In [96]:
# fill na with value above


In [97]:
lotr

,text,book,chapter,0
0,* PROLOGUE *,* PROLOGUE *,NaN,NaN
1,1. Concerning Hobbits,NaN,NaN,NaN
2,"This book is largely concerned with Hobbits, a...",NaN,NaN,NaN
3,reader may discover much of their character an...,NaN,NaN,NaN
4,Further information will also be found in the ...,NaN,NaN,NaN
...,...,...,...,...
39111,1 Every month in the Shire-calendar had 30 days.,NaN,Chapter 9 . The Grey Havens,NaN
39112,"1 See App. F, 520.",NaN,Chapter 9 . The Grey Havens,NaN
39113,( There were thirty days in March (or Rethe) i...,NaN,Chapter 9 . The Grey Havens,NaN
39114,"( It was probably Orkish in origin: sharky, 'o...",NaN,Chapter 9 . The Grey Havens,NaN
